In [118]:
import gurobipy as gp
from gurobipy import *
import pandas as pd
import numpy as np

In [119]:
# Specify data types for problematic columns
dtype_options = {'nation_position': 'str', 'nation_logo_url': 'str'}

players = pd.read_csv('players_22.csv', dtype=dtype_options)

In [120]:
players.shape

(19239, 110)

In [121]:
players.loc[:, 'pace': 'goalkeeping_speed'].isnull().sum()

pace                            2132
shooting                        2132
passing                         2132
dribbling                       2132
defending                       2132
physic                          2132
attacking_crossing                 0
attacking_finishing                0
attacking_heading_accuracy         0
attacking_short_passing            0
attacking_volleys                  0
skill_dribbling                    0
skill_curve                        0
skill_fk_accuracy                  0
skill_long_passing                 0
skill_ball_control                 0
movement_acceleration              0
movement_sprint_speed              0
movement_agility                   0
movement_reactions                 0
movement_balance                   0
power_shot_power                   0
power_jumping                      0
power_stamina                      0
power_strength                     0
power_long_shots                   0
mentality_aggression               0
m

In [122]:
#players.dropna(subset=players.loc[:, 'pace': 'goalkeeping_reflexes'].columns, inplace=True)
# We don't drop missing values because some missing values are associated with positions. For example, goalkeepers don't have scores for pace, shooting...

In [123]:
# Separate the position score columns, for example, column 'ls' with value '83+3' separates into column 'ls' with value 83 and column 'ls_potential' 3
for col in players.loc[:, 'ls': 'gk'].columns:
    # Extract the numbers using regular expressions and handle the case where the sign is not present
    extracted_values = players[col].str.extract(r'(\d+)([\+\-]?\d*)')
    # Separate the extracted values into two columns
    players[[col, col+'_potential']] = extracted_values.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    # Fill missing values with 0 in the 'Value2' column
    players[col+'_potential'].fillna(0, inplace=True)

In [124]:
players['player_positions'].isnull().sum()

0

In [125]:
# Cleaning the "player_positions" column by removing quotes
players['player_positions'] = players['player_positions'].str.replace('"', '')
players['player_positions'] = players['player_positions'].str.replace(' ', '')

# Splitting the positions and getting dummies
positions_dummies = players['player_positions'].str.get_dummies(sep=',')

# Merging the dummies back into the original dataset
players = pd.concat([players, positions_dummies], axis=1)

In [126]:
# Creating the 'forward_score' column
forward_cols = ['pace', 'shooting', 'passing', 'dribbling', 'physic', 'attacking_crossing',
                'attacking_finishing', 'attacking_heading_accuracy', 'attacking_volleys',
                'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
                'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
                'movement_agility', 'movement_reactions', 'power_shot_power', 'power_jumping',
                'power_strength', 'power_long_shots', 'mentality_aggression',
                'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                'mentality_penalties', 'mentality_composure']

players['forward_score'] = players[forward_cols].sum(axis=1)

# Creating the 'mid_score' column
mid_cols = ['pace', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing',
            'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve',
            'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_agility',
            'movement_balance', 'mentality_aggression', 'mentality_interceptions',
            'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure',
            'defending_marking_awareness']

players['mid_score'] = players[mid_cols].sum(axis=1)

# Creating the 'back_score' column
back_cols = ['passing', 'defending', 'physic', 'movement_acceleration', 'movement_sprint_speed',
             'movement_agility', 'movement_reactions', 'movement_balance', 'power_jumping',
             'power_stamina', 'power_strength', 'mentality_positioning', 'mentality_vision',
             'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle',
             'defending_sliding_tackle']

players['back_score'] = players[back_cols].sum(axis=1)

# Creating columns of 'forward', 'midfielder', 'back'
# Forward: CF, ST
# Midfielder: CAM, CDM, CM, LM, LW, LWB, RM, RW, RWB
# Back: CB, LB, RB
# One player can have up to 3 positions
players['forward'] = players[['CF', 'ST']].sum(axis=1)
players.loc[players['forward'] >= 1, 'forward'] = 1

players['midfielder'] = players[['CAM', 'CDM', 'CM', 'LM', 'LW', 'LWB', 'RM', 'RW', 'RWB']].sum(axis=1)
players.loc[players['midfielder'] >= 1, 'midfielder'] = 1

players['back'] = players[['CB', 'LB', 'RB']].sum(axis=1)
players.loc[players['back'] >= 1, 'back'] = 1

In [127]:
# forward_score, mid_score, back_score range from 0-100
from sklearn.preprocessing import MinMaxScaler
features = players.iloc[:, -6:-3] # split numerical features
scaler = MinMaxScaler()
features = scaler.fit_transform(features.values)
players.iloc[:, -6:-3] = features*100

In [128]:
#players.to_csv('~/Desktop/players.csv', index=False)

In [129]:
#pd.set_option('display.max_columns', 150)
#players.head()

In [130]:
players_data = players[players['nationality_name'] == 'China PR']
players_data.shape

(385, 158)

In [131]:
pd.set_option('display.max_columns', 150)
players_data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,...,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,ls_potential,st_potential,rs_potential,lw_potential,lf_potential,cf_potential,rf_potential,rw_potential,lam_potential,cam_potential,ram_potential,lm_potential,lcm_potential,cm_potential,rcm_potential,rm_potential,lwb_potential,ldm_potential,cdm_potential,rdm_potential,rwb_potential,lb_potential,lcb_potential,cb_potential,rcb_potential,rb_potential,gk_potential,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,forward_score,mid_score,back_score,forward,midfielder,back
536,192064,https://sofifa.com/player/192064/late-gao/220002,Gao Late,高拉特,"ST,RW,CAM",79,79,17000000.0,32000.0,30,1991-06-05,185,78,111839.0,Guangzhou FC,Chinese Super League,1.0,CAM,11.0,NaN,2020-01-01,2023.0,155,China PR,NaN,NaN,NaN,Right,4,4,2,High/Medium,Normal (185+),No,27200000.0,NaN,"Power Free-Kick, Playmaker (AI), Chip Shot (AI...",83.0,78.0,77.0,79.0,41.0,72.0,73,77,78,81,77,78,70,72,74,83,82,83,74,76,81,78,78,76,73,79,62,33,83,77,76,78,37,41,34,9,14,8,...,80,80,79,79,79,79,79,75,75,75,79,62,61,61,61,62,59,54,54,54,59,17,https://cdn.sofifa.net/players/192/064/22_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,86.780715,83.124216,76.923077,1,1,0
538,192598,https://sofifa.com/player/192598/kesen-ai/220002,Ai Kesen,艾克森,ST,79,79,15000000.0,32000.0,31,1989-07-13,180,77,111839.0,Guangzhou FC,Chinese Super League,1.0,RS,9.0,NaN,2019-07-09,2023.0,155,China PR,1413.0,ST,9.0,Right,4,4,2,High/High,Stocky (170-185),No,24000000.0,NaN,"Power Free-Kick, Chip Shot (AI), Technical Dri...",84.0,80.0,73.0,79.0,48.0,79.0,67,80,72,75,76,78,76,80,66,79,87,81,83,75,90,86,83,81,82,72,69,56,77,78,85,76,43,45,35,9,16,7,...,78,78,78,77,77,77,77,73,73,73,77,65,64,64,64,65,62,59,59,59,62,17,https://cdn.sofifa.net/players/192/598/22_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/teams/1413/60.png,https://cdn.sofifa.net/flags/cn.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,89.683774,86.637390,81.719457,1,0,0
891,188044,https://sofifa.com/player/188044/lan-a/220002,A Lan,阿兰,"ST,LW,LM",77,77,9000000.0,27000.0,31,1989-07-10,178,64,111839.0,Guangzhou FC,Chinese Super League,1.0,SUB,18.0,NaN,2015-12-01,2023.0,155,China PR,1413.0,LW,11.0,Right,3,4,2,High/Low,Lean (170-185),No,14400000.0,NaN,"Long Passer (AI), Speed Dribbler (AI)",84.0,75.0,68.0,81.0,33.0,72.0,66,79,76,68,76,83,78,72,63,79,84,84,88,75,75,74,82,77,75,68,55,33,78,72,7

In [161]:
# Initialize the model
model = gp.Model("soccer_team_optimization")
model.Params.LogToConsole = 0

# Add variables: One for each player, 1 if the player is selected for 23-player team(including substitute), otherwise 0
X = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="player1/23")

# Add variables: One for each player, 1 if the player is selected for 11-player team(excluding substitute), otherwise 0
Y = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="player1/11")

# Add variables: One for each player, 1 if the player is selected for 23-player team and is Forward/Midfielder/Back position, otherwise 0
F = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Forward")
M = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Midfielder")
B = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Back")

# Auxiliary variables for age constraint
Z = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="z")

# Objective 1: Maximize overall rating for 23 players
overall_rating_23 = gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0] 
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(overall_rating_23, index = 0, priority = 2)

# Objective 2: Maximize skill scores for each position
score_forward = gp.quicksum(F[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
score_mid = gp.quicksum(M[p] * players_data.loc[players_data['sofifa_id'] == p, 'mid_score'].values[0]
                             for p in players_data['sofifa_id'])
score_back = gp.quicksum(B[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(score_forward+score_mid+score_back, index = 1, priority = 1)

# Objective 3: Maximize overall rating for 11 players
overall_rating_11 = gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0] 
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(overall_rating_11, index = 2, priority = 0)

# Objective 4: Maximize team chemistry (number of players from the same club)
chemistry = gp.quicksum(X[p1] * X[p2] 
                        for i, p1 in enumerate(players_data['sofifa_id']) 
                        for p2 in players_data['sofifa_id'][i+1:] 
                        if players_data.loc[players_data['sofifa_id'] == p1, 'club_team_id'].values[0] == 
                           players_data.loc[players_data['sofifa_id'] == p2, 'club_team_id'].values[0])
# model.setObjectiveN(chemistry, index = 3, priority = 0)
## multi objective need to be linear

#model.ModelSense = GRB.MAXIMIZE
model.setObjective(0.4*overall_rating_23+0.3*(score_forward+score_mid+score_back)+0.2*overall_rating_11+0.1*chemistry, GRB.MAXIMIZE)

# Constraint: 23 players including substitutes
model.addConstr(X.sum() == 23)
# Constraint: 11 players excluding substitutes
model.addConstr(Y.sum() == 11)
# Constraint: Relationship between 11 and 23 players
model.addConstrs(Y[p] <= X[p] for p in players_data['sofifa_id'])
# Constraint: for 23 players, each player's age <= 40
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0] <= 40
                             for p in players_data['sofifa_id'])
# Constraint: for 23 players, average player's age <= 30, >= 28
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0]
                            for p in players_data['sofifa_id']) >= 28*23)
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0]
                            for p in players_data['sofifa_id']) <= 30*23)
# Constraint: for 23 players, at least 3 players with age <= 23 need to be selected
n = len(players_data)
bigM = 100
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0] <= 23+bigM*Z[p]
                             for p in players_data['sofifa_id'])
model.addConstr(Z.sum() <= n-3)
# Constraint: relationship between selected 23 players and their positions
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward'].values[0] >= F[p]
                             for p in players_data['sofifa_id'])
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'midfielder'].values[0] >= M[p]
                             for p in players_data['sofifa_id'])
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'back'].values[0] >= B[p]
                             for p in players_data['sofifa_id'])

# Constraint: one player is selected into the 23-player team for one position, excluding goal keepers
model.addConstrs(F[p]+M[p]+B[p] <= 1 for p in players_data['sofifa_id'])
# Position constraints: number of forwards/midfielders/backs for 23 players
model.addConstr(F.sum() >= 4)
model.addConstr(M.sum() >= 5)
model.addConstr(B.sum() >= 5)
model.addConstr(F.sum() <= 7)
model.addConstr(M.sum() <= 8)
model.addConstr(B.sum() <= 8)
# Position constraint: number of goal keepers
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0]
                            for p in players_data['sofifa_id']) == 3)
# For 23-player team, #CF > = 2 
# model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'CF'].values[0]
#                             for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #LW >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] >= p, 'LW'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #RW >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'RW'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #CAM >= 1
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'CAM'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 23-player team, #CDM >= 1
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'CDM'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 23-player team, #CB >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'CB'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #LB >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'LB'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #RB >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'RB'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 11-player team, 1 GK
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0]
                            for p in players_data['sofifa_id']) == 1)
# For 11-player team, 3-4-3
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward'].values[0]
                            for p in players_data['sofifa_id']) == 3)
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'midfielder'].values[0]
                            for p in players_data['sofifa_id']) == 4)
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'back'].values[0]
                            for p in players_data['sofifa_id']) == 3)
# For 11-player team, #LW >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] >= p, 'LW'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 11-player team, #RW >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'RW'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 11-player team, #CB >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'CB'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 11-player team, #LB >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'LB'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 11-player team, #RB >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'RB'].values[0]
                            for p in players_data['sofifa_id']) >= 1)

# Optimize the model
model.optimize()

# Get the selected players
selected_players23 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if X[p].x > 0]
selected_players11 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if Y[p].x > 0]
selected_forwards = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if F[p].x > 0]
selected_midfielders = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if M[p].x > 0]
selected_backs = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if B[p].x > 0]
selected_goalkeepers = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id']
                        if X[p].x * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0] > 0]
selected_forwards11 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if Y[p].x*F[p].x > 0]
selected_midfielders11 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if Y[p].x*M[p].x > 0]
selected_backs11 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if Y[p].x*B[p].x > 0]
selected_goalkeeper11 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id']
                        if Y[p].x * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0] > 0]

print("Selected 23 Players:", selected_players23)
print("Selected Forwards:", selected_forwards)
print("Selected Midfielders:", selected_midfielders)
print("Selected Backs:", selected_backs)
print("Selected Goalkeepers:", selected_goalkeepers)
print("")
print("Selected 11 Players:", selected_players11)
print("Selected Forwards:", selected_forwards11)
print("Selected Midfielders:", selected_midfielders11)
print("Selected Backs:", selected_backs11)
print("Selected Goalkeeper:", selected_goalkeeper11)

Selected 23 Players: ['Gao Late', 'Ai Kesen', 'A Lan', 'Fei Nanduo', 'Wu Lei', 'Yan Junling', 'Cao Yunding', 'Wang Dalei', 'Hao Junmin', 'Yu Hanchao', 'Li Xuepeng', 'Zhang Linpeng', 'Huang Bowen', 'Ji Xiang', 'Jiang Zhipeng', 'Li Ke', 'Wei Shihao', 'Liu Dianzuo', 'Deng Hanwen', 'Gao Zhunyi', 'Wang Shangyuan', 'Zhang Yuning', 'Dong Xuesheng']
Selected Forwards: ['Gao Late', 'Ai Kesen', 'A Lan', 'Wu Lei', 'Wei Shihao', 'Zhang Yuning', 'Dong Xuesheng']
Selected Midfielders: ['Fei Nanduo', 'Cao Yunding', 'Hao Junmin', 'Yu Hanchao', 'Huang Bowen', 'Ji Xiang', 'Jiang Zhipeng', 'Li Ke']
Selected Backs: ['Li Xuepeng', 'Zhang Linpeng', 'Deng Hanwen', 'Gao Zhunyi', 'Wang Shangyuan']
Selected Goalkeepers: ['Yan Junling', 'Wang Dalei', 'Liu Dianzuo']

Selected 11 Players: ['Ai Kesen', 'Fei Nanduo', 'Yan Junling', 'Cao Yunding', 'Hao Junmin', 'Yu Hanchao', 'Li Xuepeng', 'Zhang Linpeng', 'Deng Hanwen', 'Zhang Yuning', 'Dong Xuesheng']
Selected Forwards: ['Ai Kesen', 'Zhang Yuning', 'Dong Xuesheng']


In [158]:
## Set the Korean team formation
korean_players = players[players['nationality_name'] == 'Korea Republic']
korean_players_11 = korean_players[korean_players['short_name'].isin(['H. Son', 'Kim Min Jae', 'Kim Seung Gyu', 'Lee Jae Sung', 'Kangin Lee', 'Lee Ki Je', 'Hwang Hee Chan', 'Jeong Woo Yeong', 'Hwang Ui Jo', 'Seol Young Woo', 'Kim Young Gwon'])]

korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', ['CF', 'LW']] = 0
korean_players_11.loc[korean_players_11['short_name'] == 'Hwang Ui Jo', ['LM']] = 0
korean_players_11.loc[korean_players_11['short_name'] == 'Lee Jae Sung', ['CF', 'CAM']] = 0
korean_players_11.loc[korean_players_11['short_name'] == 'Kangin Lee', ['ST', 'RM']] = 0
korean_players_11.loc[korean_players_11['short_name'] == 'Jeong Woo Yeong', ['CF']] = 0
korean_players_11.loc[korean_players_11['short_name'] == 'Lee Ki Je', ['LM', 'LWB']] = 0
korean_players_11.loc[korean_players_11['short_name'] == 'Lee Ki Je', ['LB']] = 1
korean_players_11.loc[korean_players_11['short_name'] == 'Seol Young Woo', ['LM', 'LB']] = 0
korean_players_11.loc[korean_players_11['short_name'] == 'Seol Young Woo', ['RB']] = 1
korean_players_11['forward'] = korean_players_11[['CF', 'ST']].sum(axis=1)
korean_players_11['midfielder'] = korean_players_11[['CAM', 'CDM', 'CM', 'LM', 'LW', 'LWB', 'RM', 'RW', 'RWB']].sum(axis=1)
korean_players_11['back'] = korean_players_11[['CB', 'LB', 'RB']].sum(axis=1)

korean_players_11

/var/folders/m4/wpp5ytyx7cx5d0jgdsbgt_3r0000gn/T/ipykernel_5820/1783767553.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  korean_players_11['forward'] = korean_players_11[['CF', 'ST']].sum(axis=1)
/var/folders/m4/wpp5ytyx7cx5d0jgdsbgt_3r0000gn/T/ipykernel_5820/1783767553.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  korean_players_11['midfielder'] = korean_players_11[['CAM', 'CDM', 'CM', 'LM', 'LW', 'LWB', 'RM', 'RW', 'RWB']].sum(axis=1)
/var/folders/m4/wpp5ytyx7cx5d0jgdsbgt_3r0000gn/T/ipykerne

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,ls_potential,st_potential,rs_potential,lw_potential,lf_potential,cf_potential,rf_potential,rw_potential,lam_potential,cam_potential,ram_potential,lm_potential,lcm_potential,cm_potential,rcm_potential,rm_potential,lwb_potential,ldm_potential,cdm_potential,rdm_potential,rwb_potential,lb_potential,lcb_potential,cb_potential,rcb_potential,rb_potential,gk_potential,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,forward_score,mid_score,back_score,forward,midfielder,back
13,200104,https://sofifa.com/player/200104/heung-min-son...,H. Son,손흥민 孙兴慜,"LM,CF,LW",89,89,104000000.0,220000.0,28,1992-07-08,183,78,18.0,Tottenham Hotspur,English Premier League,1.0,LW,7.0,NaN,2015-08-28,2025.0,167,Korea Republic,NaN,NaN,NaN,Right,5,4,4,High/High,Unique,Yes,197600000.0,"#Dribbler, #Engine, #Distance Shooter, #Acroba...","Solid Player, Finesse Shot, Long Shot Taker (A...",88.0,87.0,82.0,86.0,43.0,69.0,83,88,68,84,78,87,85,74,74,84,85,90,86,91,78,88,60,88,64,89,62,39,91,83,73,89,50,34,33,11,13,13,6,10,NaN,85,85,85,87,87,87,87,87,86,86,86,86,80,80,80,86,68,64,64,64,68,64,54,54,54,64,19,https://cdn.sofifa.net/players/200/104/22_120.png,https://cdn.sofifa.net/teams/18/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/kr.png,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,93.001555,89.836888,81.719457,0,1,0
1286,212260,https://sofifa.com/player/212260/ui-jo-hwang/2...,Hwang Ui Jo,황의조 Ui Jo Hwang,"ST,LM",76,76,7500000.0,29000.0,28,1992-08-28,185,82,59.0,FC Girondins de Bordeaux,French Ligue 1,1.0,LS,18.0,NaN,2019-07-18,2023.0,167,Korea Republic,NaN,NaN,NaN,Right,4,3,2,High/Medium,Lean (185+),No,15000000.0,NaN,"Finesse Shot, Long Shot Taker (AI), Outside Fo...",78.0,77.0,64.0,74.0,42.0,73.0,62,78,71,67,73,74,69,63,57,75,80,77,77,76,64,77,74,80,73,77,62,44,79,66,66,69,33,42,36,9,15,10,7,6,NaN,76,76,76,74,75,75,75,74,73,73,73,72,68,68,68,72,60,57,57,57,60,58,54,54,54,58,17,https://cdn.sofifa.net/players/212/260/22_120.png,https://cdn.sofifa.net/teams/59/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/kr.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,80.663556,74.968632,71.674208,1,0,0
1752,221671,https://sofifa.com/player/221671/jae-sung-lee/...,Lee Jae Sung,이재성 李在成,"CM,CF,CAM",75,75,5500000.0,24000.0,28,1992-08-10,180,70,169.0,1. FSV Mainz 05,German 1. Bundesliga,1.0,SUB,7.0,NaN,2021-07-08,2024.0,167,Korea Republic,NaN,NaN,NaN,Left,4,3,2,High/High,Lean (17

In [179]:
#### How to arrange positions for selected 11 players against Korean team
chinese_players11 = players_data[players_data['short_name'].isin(selected_players11)]
# Goalkeeper has been decided, so we don't consider goalkeeper
chinese_players10 = chinese_players11.drop(index=chinese_players11[chinese_players11['short_name'].isin(selected_goalkeeper11)].index)
# Initialize the model
model = gp.Model("Chinese_soccer_team_optimization")
model.Params.LogToConsole = 0

# Add variables: One for each player, 1 if the player is selected as ST/CF/LW/CM/RW/LCB/CB/RCB, otherwise 0
ST = model.addVars(chinese_players10['sofifa_id'], vtype=GRB.BINARY, name="Striker")
CF = model.addVars(chinese_players10['sofifa_id'], vtype=GRB.BINARY, name="Center Forward")
LW = model.addVars(chinese_players10['sofifa_id'], vtype=GRB.BINARY, name="Left Winger")
CM = model.addVars(chinese_players10['sofifa_id'], vtype=GRB.BINARY, name="Center Midfielder")
RW = model.addVars(chinese_players10['sofifa_id'], vtype=GRB.BINARY, name="Right Winger")
LCB = model.addVars(chinese_players10['sofifa_id'], vtype=GRB.BINARY, name="Left Central Back")
CB = model.addVars(chinese_players10['sofifa_id'], vtype=GRB.BINARY, name="Center Back")
RCB = model.addVars(chinese_players10['sofifa_id'], vtype=GRB.BINARY, name="Right Central Back")

# Add variables: gap between H.Son and our LW
# Gap between physic
gap1 = model.addVar(vtype=GRB.CONTINUOUS, name="Physic Gap")
# Gap between movement_balance
gap2 = model.addVar(vtype=GRB.CONTINUOUS, name="Movement Balance Gap")
# Gap between power_jumping
gap3 = model.addVar(vtype=GRB.CONTINUOUS, name="Power Jumping Gap")
# Gap between power_strength
gap4 = model.addVar(vtype=GRB.CONTINUOUS, name="Power Strength Gap")
# Gap between pace
gap5 = model.addVar(vtype=GRB.CONTINUOUS, name="Pace Gap")
# Gap between Son mentality_positioning and our defending_marking_awareness
gap6 = model.addVar(vtype=GRB.CONTINUOUS, name="Defending Marking Awareness Gap")
# Gap between Son mentality_positioning and our defending_standing_tackle
gap7 = model.addVar(vtype=GRB.CONTINUOUS, name="Defending Standing Tackle Gap")
# Gap between Son skill_dribbling and our defending_sliding_tackle
gap8 = model.addVar(vtype=GRB.CONTINUOUS, name="Defending Sliding Tackle Gap")

# Calculate position scores
st_scores = gp.quicksum(ST[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'st'].values[0]
                             for p in chinese_players10['sofifa_id'])
cf_scores = gp.quicksum(CF[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'cf'].values[0]
                             for p in chinese_players10['sofifa_id'])
lw_scores = gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'lw'].values[0]
                             for p in chinese_players10['sofifa_id'])
cm_scores = gp.quicksum(CM[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'cm'].values[0]
                             for p in chinese_players10['sofifa_id'])
rw_scores = gp.quicksum(RW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'rw'].values[0]
                             for p in chinese_players10['sofifa_id'])
lcb_scores = gp.quicksum(LCB[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'lcb'].values[0]
                             for p in chinese_players10['sofifa_id'])
cb_scores = gp.quicksum(CB[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'cb'].values[0]
                             for p in chinese_players10['sofifa_id'])
rcb_scores = gp.quicksum(RCB[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'rcb'].values[0]
                             for p in chinese_players10['sofifa_id'])
# Maximize position scores for forwards, midfielders, backs
model.setObjective(st_scores+cf_scores+lw_scores+cm_scores+rw_scores+lcb_scores+cb_scores+rcb_scores
-5*(gap1+gap2+gap3+gap4+gap5)+5*(gap6+gap7+gap8), GRB.MAXIMIZE)

# Constraint: 2 strikers
model.addConstr(ST.sum() == 2)
# Constraint: 1 center forward
model.addConstr(CF.sum() == 1)
# Constraint: 1 left winger
model.addConstr(LW.sum() == 1)
# Constraint: 2 center midfielder
model.addConstr(CM.sum() == 2)
# Constraint: 1 right winger
model.addConstr(RW.sum() == 1)
# Constraint: 1 left central back
model.addConstr(LCB.sum() == 1)
# Constraint: 1 center back
model.addConstr(CB.sum() == 1)
# Constraint: 1 right central back
model.addConstr(RCB.sum() == 1)

# Constraint: one player for one position
model.addConstrs(ST[p]+CF[p]+LW[p]+CM[p]+RW[p]+LCB[p]+CB[p]+RCB[p] == 1 for p in chinese_players10['sofifa_id'])

# Constraint: Players are assigned to their positions (align with player_position)
# model.addConstr(gp.quicksum(ST[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'ST'].values[0]
#                              for p in chinese_players10['sofifa_id']) == 2)
# model.addConstr(gp.quicksum(CF[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'CF'].values[0]
#                              for p in chinese_players10['sofifa_id']) == 1)
# model.addConstr(gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'LW'].values[0]
#                              for p in chinese_players10['sofifa_id']) == 1)
# model.addConstr(gp.quicksum(CM[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'CM'].values[0]
#                              for p in chinese_players10['sofifa_id']) == 2)
# model.addConstr(gp.quicksum(RW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'RW'].values[0]
#                              for p in chinese_players10['sofifa_id']) == 1)
# model.addConstr(gp.quicksum(LCB[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'CB'].values[0]
#                              for p in chinese_players10['sofifa_id']) == 1)
# model.addConstr(gp.quicksum(CB[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'CB'].values[0]
#                              for p in chinese_players10['sofifa_id']) == 1)
# model.addConstr(gp.quicksum(RCB[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'CB'].values[0]
#                              for p in chinese_players10['sofifa_id']) == 1)

## Constraint: Gap
# Gap between physic
model.addConstr(gap1 >= korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'physic'].values[0]-
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'physic'].values[0]
                             for p in chinese_players10['sofifa_id']))
model.addConstr(gap1 >= -korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'physic'].values[0]+
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'physic'].values[0]
                             for p in chinese_players10['sofifa_id']))
# Gap between movement_balance
model.addConstr(gap2 >= korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'movement_balance'].values[0]-
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'movement_balance'].values[0]
                             for p in chinese_players10['sofifa_id']))
model.addConstr(gap2 >= -korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'movement_balance'].values[0]+
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'movement_balance'].values[0]
                             for p in chinese_players10['sofifa_id']))
# Gap between power_jumping
model.addConstr(gap3 >= korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'power_jumping'].values[0]-
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'power_jumping'].values[0]
                             for p in chinese_players10['sofifa_id']))
model.addConstr(gap3 >= -korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'power_jumping'].values[0]+
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'power_jumping'].values[0]
                             for p in chinese_players10['sofifa_id']))
# Gap between power_strength
model.addConstr(gap4 >= korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'power_strength'].values[0]-
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'power_strength'].values[0]
                             for p in chinese_players10['sofifa_id']))
model.addConstr(gap4 >= -korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'power_strength'].values[0]+
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'power_strength'].values[0]
                             for p in chinese_players10['sofifa_id']))
# Gap between pace
model.addConstr(gap5 >= korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'pace'].values[0]-
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'pace'].values[0]
                             for p in chinese_players10['sofifa_id']))
model.addConstr(gap5 >= -korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'pace'].values[0]+
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'pace'].values[0]
                             for p in chinese_players10['sofifa_id']))
# Gap between Son mentality_positioning and our defending_marking_awareness
model.addConstr(gap6 >= korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'mentality_positioning'].values[0]-
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'defending_marking_awareness'].values[0]
                             for p in chinese_players10['sofifa_id']))
model.addConstr(gap6 >= -korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'mentality_positioning'].values[0]+
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'defending_marking_awareness'].values[0]
                             for p in chinese_players10['sofifa_id']))
# Gap between Son mentality_positioning and our defending_standing_tackle
model.addConstr(gap7 >= korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'mentality_positioning'].values[0]-
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'defending_standing_tackle'].values[0]
                             for p in chinese_players10['sofifa_id']))
model.addConstr(gap7 >= -korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'mentality_positioning'].values[0]+
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'defending_standing_tackle'].values[0]
                             for p in chinese_players10['sofifa_id']))
# Gap between Son skill_dribbling and our defending_sliding_tackle
model.addConstr(gap8 >= korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'skill_dribbling'].values[0]-
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'defending_sliding_tackle'].values[0]
                             for p in chinese_players10['sofifa_id']))
model.addConstr(gap8 >= -korean_players_11.loc[korean_players_11['short_name'] == 'H. Son', 'skill_dribbling'].values[0]+
                        gp.quicksum(LW[p] * chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'defending_sliding_tackle'].values[0]
                             for p in chinese_players10['sofifa_id']))

model.optimize()

selected_ST = [chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'short_name'].values[0] for p in chinese_players10['sofifa_id'] if ST[p].x > 0]
selected_CF = [chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'short_name'].values[0] for p in chinese_players10['sofifa_id'] if CF[p].x > 0]
selected_LW = [chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'short_name'].values[0] for p in chinese_players10['sofifa_id'] if LW[p].x > 0]
selected_CM = [chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'short_name'].values[0] for p in chinese_players10['sofifa_id'] if CM[p].x > 0]
selected_RW = [chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'short_name'].values[0] for p in chinese_players10['sofifa_id'] if RW[p].x > 0]
selected_LCB = [chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'short_name'].values[0] for p in chinese_players10['sofifa_id'] if LCB[p].x > 0]
selected_CB = [chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'short_name'].values[0] for p in chinese_players10['sofifa_id'] if CB[p].x > 0]
selected_RCB = [chinese_players10.loc[chinese_players10['sofifa_id'] == p, 'short_name'].values[0] for p in chinese_players10['sofifa_id'] if RCB[p].x > 0]

print("Selected Strikers", selected_ST)
print("Selected Center Forward", selected_CF)
print("Selected Left Winger", selected_LW)
print("Selected Center Midfielders", selected_CM)
print("Selected Right Winger", selected_RW)
print("Selected Left Central Back", selected_LCB)
print("Selected Center Back", selected_CB)
print("Selected Right Central Back", selected_RCB)
print("Selected Goalkeeper", selected_goalkeeper11)

Selected Strikers ['Zhang Yuning', 'Dong Xuesheng']
Selected Center Forward ['Hao Junmin']
Selected Left Winger ['Cao Yunding']
Selected Center Midfielders ['Ai Kesen', 'Fei Nanduo']
Selected Right Winger ['Yu Hanchao']
Selected Left Central Back ['Li Xuepeng']
Selected Center Back ['Deng Hanwen']
Selected Right Central Back ['Zhang Linpeng']
Selected Goalkeeper ['Yan Junling']


In [163]:
pd.set_option('display.max_columns', 200)
chinese_players11

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,ls_potential,st_potential,rs_potential,lw_potential,lf_potential,cf_potential,rf_potential,rw_potential,lam_potential,cam_potential,ram_potential,lm_potential,lcm_potential,cm_potential,rcm_potential,rm_potential,lwb_potential,ldm_potential,cdm_potential,rdm_potential,rwb_potential,lb_potential,lcb_potential,cb_potential,rcb_potential,rb_potential,gk_potential,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,forward_score,mid_score,back_score,forward,midfielder,back
538,192598,https://sofifa.com/player/192598/kesen-ai/220002,Ai Kesen,艾克森,ST,79,79,15000000.0,32000.0,31,1989-07-13,180,77,111839.0,Guangzhou FC,Chinese Super League,1.0,RS,9.0,NaN,2019-07-09,2023.0,155,China PR,1413.0,ST,9.0,Right,4,4,2,High/High,Stocky (170-185),No,24000000.0,NaN,"Power Free-Kick, Chip Shot (AI), Technical Dri...",84.0,80.0,73.0,79.0,48.0,79.0,67,80,72,75,76,78,76,80,66,79,87,81,83,75,90,86,83,81,82,72,69,56,77,78,85,76,43,45,35,9,16,7,8,7,NaN,79,79,79,78,78,78,78,78,77,77,77,77,73,73,73,77,65,64,64,64,65,62,59,59,59,62,17,https://cdn.sofifa.net/players/192/598/22_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/teams/1413/60.png,https://cdn.sofifa.net/flags/cn.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,89.683774,86.637390,81.719457,1,0,0
1278,211108,https://sofifa.com/player/211108/nanduo-fei/22...,Fei Nanduo,费南多,"RM,LW,CAM",76,76,7500000.0,24000.0,28,1993-03-16,174,65,111839.0,Guangzhou FC,Chinese Super League,1.0,RM,19.0,NaN,2020-01-01,2023.0,155,China PR,NaN,NaN,NaN,Left,3,4,1,High/Low,Lean (170-185),No,12000000.0,NaN,"Speed Dribbler (AI), Technical Dribbler (AI)",85.0,71.0,74.0,75.0,39.0,68.0,70,70,44,74,62,80,67,74,75,67,84,85,79,77,72,75,75,76,66,71,63,36,73,78,63,69,32,46,42,6,11,11,10,8,NaN,70,70,70,75,74,74,74,75,75,75,75,75,71,71,71,75,61,59,59,59,61,58,51,51,51,58,16,https://cdn.sofifa.net/players/211/108/22_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,80.196993,77.728984,74.389140,0,1,0
2799,224599,https://sofifa.com/player/224599/junling-yan/2...,Yan Junling,颜骏凌,GK,73,73,2000000.0,14000.0,30,1991-01-28,191,83,112540.0,Shanghai Port FC,Chinese Super League,1.0,GK,1.0,NaN,2013-03-08,2025.0,155,China PR,1413.0,GK,1.0,Right,3,1,1,Medium/Medium,Normal (185+),No,3100000.0,NaN,"Solid Player, Team Player",Na